In [1]:
import json
import numpy as np
import random
import cv2
import matplotlib.pyplot as plt
import mxnet as mx
import os, urllib
from collections import namedtuple
%matplotlib inline

In [2]:
def get_list():
    json_dir = '/data3/srip_face/diving/code/Label_review/result/'
    f_lst = []
    for itm in os.listdir(json_dir):
        if itm.startswith('seg') and itm != 'seg_006.json':
            f_lst.append(json_dir+itm)
    lst = []
    for itm in f_lst:
        with open(itm, 'r') as f:
            lst.extend(json.load(f))
    random.seed(1234)
    random.shuffle(lst)
    return lst

In [3]:
def load_model(s_fname, p_fname):
    """
    Load model checkpoint from file.
    :return: (arg_params, aux_params)
    arg_params : dict of str to NDArray
        Model parameter, dict of name to NDArray of net's weights.
    aux_params : dict of str to NDArray
        Model parameter, dict of name to NDArray of net's auxiliary states.
    """
    symbol = mx.symbol.load(s_fname)
    save_dict = mx.nd.load(p_fname)
    arg_params = {}
    aux_params = {}
    for k, v in save_dict.items():
        tp, name = k.split(':', 1)
        if tp == 'arg':
            arg_params[name] = v
        if tp == 'aux':
            aux_params[name] = v
    return symbol, arg_params, aux_params

model_symbol = "caffenet-mxnet-jump-start-128-symbol.json"
model_params = "caffenet-mxnet-jump-start-128-0005.params"
sym, arg_params, aux_params = load_model(model_symbol, model_params)
mod = mx.mod.Module(symbol=sym)

# bind the model and set training == False; Define the data shape
dshape = [('data', (1,3,224,224))]
mod.bind(for_training=False, data_shapes=dshape)
mod.set_params(arg_params, aux_params)

/home/srip_face/.local/lib/python2.7/site-packages/mxnet/module/base_module.py:65: UserWarning: Data provided by label_shapes don't match names specified by label_names ([] vs. ['softmax_label'])
  warnings.warn(msg)


In [4]:
lst = get_list()
t = int(0.8*len(lst))
print t
train_lst = lst[0:t]

260


In [5]:
def get_start_time(lst):
    lst2 = []
    for itm in lst:
        lst2.append(itm['time'][0])
    return lst2

In [6]:
def time_check(c, time, fps):
    t = c*1.0/fps
    for idx,itm in enumerate(time):
        if idx==0 and t>itm-3 and t<itm+1.5:
            return False
        if idx!=0 and t>itm-0.5 and t<itm+1:
            return False
    return True

In [7]:
Batch = namedtuple('Batch', ['data'])
THRESHOLD = 0.6

def preprocess_image(img, show_img=False):
    '''
    convert the image to a numpy array
    '''
    img = cv2.resize(img, (224, 224))
    img = np.swapaxes(img, 0, 2)
    img = np.swapaxes(img, 1, 2) 
    img = img[np.newaxis, :] 
    return img

total_list = []
indx = 0
for idex,itm in enumerate(train_lst):
    v_dir = '/data3/yingwei/vidDB/diving_share/firstphase/video_first10/' + itm[0] + '.mp4'
    time = get_start_time(itm[1])
    cap = cv2.VideoCapture(v_dir)
    t_list = []
    if cap.isOpened():
        fps = cap.get(5)
        step = int(fps/4.0)
        c = 0
        cap.set(1,0)
        while True:
            ret = cap.grab()
            if not ret:
                break
            if c%step != 0:
                c += 1
                continue
            ret, frame = cap.retrieve()
            if not ret:
                break
            img = preprocess_image(frame)
            mod.forward(Batch([mx.nd.array(img)]))

            # predict
            prob = mod.get_outputs()[0].asnumpy()
            score = prob[0][1].copy()
            if score>THRESHOLD and time_check(c, time, fps):
                t_list.append(c)
                cv2.imwrite('/data4/srip_face/img/jump_detector/train/0bg/bootstrap_' + format(indx, '05d') + '.jpg',frame)
                indx += 1
            c += 1
        total_list.append(t_list)
print indx

12510


In [8]:
n_fr_ = 0
for itm in total_list:
    n_fr_ += len(itm)
print n_fr_

12510
